In [ ]:
#filename ="stock_nifty_next50.csv"
filename ="stock_nifty50_365.csv"
#filename ="stock.csv"
import pandas as pd

# Load the Excel file
df = pd.read_csv("../../inputdata/trading/{filename}")


In [50]:



#stragety 1




# Prepare date and sort
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(['SYMBOL', 'TIMESTAMP']).reset_index(drop=True)

results = []

for symbol in df['SYMBOL'].unique():
    sdf = df[df['SYMBOL'] == symbol].copy()
    sdf = sdf.sort_values('TIMESTAMP').reset_index(drop=True)
    sdf['Prev_Close'] = sdf['CLOSING_PRICE'].shift(1)
    sdf['Up'] = sdf['CLOSING_PRICE'] > sdf['Prev_Close']
    sdf['Up_1'] = sdf['Up'].shift(1)
    sdf['Buy_Signal'] = sdf['Up'] & sdf['Up_1']
    profit = 0
    trades = 0
    for i, row in sdf[sdf['Buy_Signal']].iterrows():
        buy_idx = i
        sell_idx = i + 2
        if sell_idx < len(sdf):
            buy_price = sdf.loc[buy_idx, 'OPENING_PRICE']
            sell_price = sdf.loc[sell_idx, 'CLOSING_PRICE']
            trade_profit = (sell_price - buy_price) / buy_price * 100000  # ₹1 lakh per trade
            profit += trade_profit
            trades += 1
    results.append({'SYMBOL': symbol, 'Total Profit (₹)': profit, 'Trades': trades})

results_df = pd.DataFrame(results)
results_df.to_csv("../../ouput/tading/{filename}", index=False)
results_df = results_df.sort_values('Total Profit (₹)', ascending=False)
print(results_df.head(10))  # Show top 10
print("\nMost profitable symbol:", results_df.iloc[0]['SYMBOL'], "with profit ₹", round(results_df.iloc[0]['Total Profit (₹)'],2))


          SYMBOL  Total Profit (₹)  Trades
735      IFBAGRO     105619.610032       6
540     FMGOETZE      81929.500700       8
1387        RHFL      74195.791424       6
1833      WELINV      71863.512074       5
371   CREATIVEYE      71356.526945       4
478   EIMCOELECO      67908.562648       5
933     KREBSBIO      67843.912803       5
1526      SIGIND      62474.212777       5
991    LUMAXTECH      59589.602024       5
1693         TIL      58332.835665       7

Most profitable symbol: IFBAGRO with profit ₹ 105619.61


In [1]:

#starategy 2


df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(['SYMBOL', 'TIMESTAMP']).reset_index(drop=True)

results = []

# Process each symbol separately
for symbol in df['SYMBOL'].unique():
    sdf = df[df['SYMBOL'] == symbol].copy()
    sdf = sdf.sort_values('TIMESTAMP').reset_index(drop=True)
    
    # Calculate previous close and up days
    sdf['Prev_Close'] = sdf['CLOSING_PRICE'].shift(1)
    sdf['Up'] = sdf['CLOSING_PRICE'] > sdf['Prev_Close'] # Up by at least 1% 
    sdf['Up_1'] = sdf['Up'].shift(1)
    
    # 10-day average volume
    sdf['Avg_Vol_10'] = sdf['TOT_TRADED_QTY'].rolling(window=10).mean()
    
    # Buy signal: 2 consecutive up closes + volume surge (>1.2x avg volume)
    sdf['Buy_Signal'] = (sdf['Up'] & sdf['Up_1']) & (sdf['TOT_TRADED_QTY'] > 1.2 * sdf['Avg_Vol_10'])

    profit = 0
    trades = 0
    for i, row in sdf[sdf['Buy_Signal']].iterrows():
        # Buy on the next day and hold for 3 days
        print(f"Processing symbol: {symbol}, Buy signal at date {sdf.loc[i, 'TIMESTAMP']}")
        buy_idx = i
        sell_idx = i + 3  # Hold 3 days
        if sell_idx < len(sdf):
            buy_price = sdf.loc[buy_idx, 'OPENING_PRICE']
            sell_price = sdf.loc[sell_idx, 'CLOSING_PRICE']
            trade_profit = (sell_price - buy_price) / buy_price * 100000  # ₹1 lakh invested
            profit += trade_profit
            trades += 1
    
    if trades > 0:
        results.append({'SYMBOL': symbol, 'Total Profit (₹)': profit, 'Trades': trades})

# Sort by total profit descending
results_df = pd.DataFrame(results).sort_values('Total Profit (₹)', ascending=False)
results_df.to_csv("../../ouput/tading/{filename}", index=False)
print(results_df.head(10))  # Top 10 symbols by profit

best = results_df.iloc[0]

print(f"\nMost profitable symbol: {best['SYMBOL']} with total profit ₹{round(best['Total Profit (₹)'],2)} from {best['Trades']} trades")


#sum_profit = results_df['Total Profit (₹)'].sum()
print("\nTotal profit across all symbols: ₹", round(results_df['Total Profit (₹)'].sum(), 2))


Processing symbol: 360ONE, Buy signal at date 2025-06-05 00:00:00
Processing symbol: 3IINFOLTD, Buy signal at date 2025-06-09 00:00:00
Processing symbol: 3PLAND, Buy signal at date 2025-06-05 00:00:00
Processing symbol: 5PAISA, Buy signal at date 2025-06-06 00:00:00
Processing symbol: 5PAISA, Buy signal at date 2025-06-09 00:00:00
Processing symbol: AAATECH, Buy signal at date 2025-06-05 00:00:00
Processing symbol: AAATECH, Buy signal at date 2025-06-06 00:00:00
Processing symbol: AADHARHFC, Buy signal at date 2025-06-06 00:00:00
Processing symbol: AADHARHFC, Buy signal at date 2025-06-09 00:00:00
Processing symbol: AAKASH, Buy signal at date 2025-06-09 00:00:00
Processing symbol: AARTIPHARM, Buy signal at date 2025-06-04 00:00:00
Processing symbol: AAVAS, Buy signal at date 2025-06-06 00:00:00
Processing symbol: AAVAS, Buy signal at date 2025-06-09 00:00:00
Processing symbol: ABB, Buy signal at date 2025-06-09 00:00:00
Processing symbol: ABBOTINDIA, Buy signal at date 2025-06-03 00:00